In [1]:
#!pip install --upgrade oct2py
import numpy as np
from oct2py import octave
import matplotlib.pyplot as plt
import os
import pandas as pd
octave.addpath('C:/dynare/6.1/matlab')  
#octave.addpath('/opt/homebrew/opt/dynare/lib/dynare/matlab')

    _pyeval at line 57 column 30

    _pyeval at line 57 column 30



'.;C:\\Users\\blankenw\\AppData\\Local\\anaconda3\\Lib\\site-packages\\oct2py;C:\\Users\\blankenw\\AppData\\Local\\anaconda3\\Lib\\site-packages\\octave_kernel;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\lib\\octave\\9.1.0\\site\\oct\\x86_64-w64-mingw32;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\lib\\octave\\site\\oct\\api-v59\\x86_64-w64-mingw32;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\lib\\octave\\site\\oct\\x86_64-w64-mingw32;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\9.1.0\\site\\m;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\site\\api-v59\\m;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\site\\m;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\site\\m\\once_only;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\site\\m\\startup;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\lib\\octave\\9.1.0\\oct\\x86_64-w64-mingw32;C:\\Program Fi

In [2]:
n_goods=3

In [3]:
#from MyPy import DyPy

In [4]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=10000, irf=100);")

In [5]:
%run make_model.ipynb
%run make_vars.ipynb
%run make_parameters_and_shocks.ipynb
%run make_steady_state.ipynb


Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [6]:
octave.dynare('model_1.mod')
oo = octave.pull('oo_')
mm = octave.pull('M_')

Starting Dynare (version 6.1).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 12 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
7 block(s) found:
  6 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 6 equation(s)
                                 and 6 feedback variable(s).
Computing dynamic model derivatives (order 1).
Normalizing the dynamic model...
Finding the optimal block decomposition of the dynamic model...
2 block(s) found:
  1 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 6 equation(s)
                                 and 6 feedback variable(s).
Preprocessing completed.
Preprocessing time: 0h00m01s.

STEADY-STATE RESULTS:

c      		 0.291993
i      		 0.596816
k      		 1.88737
y      		 0.660578
n0     		 -0.488795
s0     		 -0.4887

In [7]:
names1 = [attr for attr in dir(oo) if not attr.startswith('__')]

Now we want to create a class where each each attribute in oo becomes an attribute of this class. So far we take only one item from mm. This is a list of endogenous variables. Below we define the class. 

In [8]:
class from_octave:
	def __init__(self, oo_structure, mm_structure, attribute_names):
		self.oo_structure = oo_structure
		self.mm_structure = mm_structure
		self. attribute_names =  attribute_names
		self.create_attributes()
		self.make_list()
     
	def create_attributes(self): 
		for attr in self.attribute_names:
			setattr(self, attr, getattr(self.oo_structure, attr)) #this is the attributes we pull from the oo structure. We are making them attributes of the class
	
	def make_list(self): 
		self.names = list(getattr(self.mm_structure, 'endo_names')) #getting one item from the mm structure. This is the endo_names
		self.names = [item[0] for item in self.names] #pulling mm gives us a list of cell arrays. We are pulling the first item from each cell array

We see that we now can reference any of the data from `oo` with the `tt.` notation. Some examples are below. 

In [9]:
tt = from_octave(oo, mm, names1)

In [10]:
tt.var

array([[ 3.82441173e-02,  2.70324425e-02,  3.23760604e-02,
         4.04176287e-02, -4.99723515e-03,  1.23850252e-04,
         2.41174117e-03, -5.89811165e-05, -1.70414960e-04,
         3.84067991e-02,  1.96363991e-03,  7.95008611e-04],
       [ 2.70324425e-02,  3.24863791e-02,  2.36110216e-02,
         3.54796154e-02, -1.84767265e-03,  7.76521036e-03,
         5.28814959e-03,  5.81264743e-04,  5.10184001e-04,
         2.80947911e-02,  2.04945744e-03, -1.61089308e-04],
       [ 3.23760604e-02,  2.36110216e-02,  3.71724721e-02,
         3.45912741e-02, -4.07107786e-03,  3.75384803e-03,
         4.34187529e-03,  5.22061719e-04,  4.40981993e-04,
         2.07165766e-02,  2.65453533e-03,  1.26431215e-03],
       [ 4.04176287e-02,  3.54796154e-02,  3.45912741e-02,
         4.62844846e-02, -4.41107200e-03,  4.09680954e-03,
         4.39984048e-03,  2.59455904e-04,  1.45658733e-04,
         4.10789168e-02,  2.41692776e-03,  4.66706524e-04],
       [-4.99723515e-03, -1.84767265e-03, -4.0710778

In [11]:
tt.steady_state

array([[ 0.29199347],
       [ 0.59681646],
       [ 1.88736536],
       [ 0.66057787],
       [-0.4887948 ],
       [-0.4887948 ],
       [ 0.35      ],
       [ 0.35      ],
       [ 0.35      ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [12]:
tt.irfs

{'c_theta0shock': array([[-2.91181890e-03, -7.62035819e-03, -1.13593179e-02,
         -1.42754042e-02, -1.64956903e-02, -1.81300746e-02,
         -1.92734439e-02, -2.00075756e-02, -2.04028100e-02,
         -2.05195192e-02, -2.04093961e-02, -2.01165867e-02,
         -1.96786809e-02, -1.91275828e-02, -1.84902708e-02,
         -1.77894632e-02, -1.70442008e-02, -1.62703539e-02,
         -1.54810664e-02, -1.46871417e-02, -1.38973793e-02,
         -1.31188673e-02, -1.23572362e-02, -1.16168799e-02,
         -1.09011459e-02, -1.02125012e-02, -9.55267399e-03,
         -8.92277734e-03, -8.32341440e-03, -7.75476931e-03,
         -7.21668482e-03, -6.70872873e-03, -6.23025029e-03,
         -5.78042821e-03, -5.35831123e-03, -4.96285224e-03,
         -4.59293692e-03, -4.24740766e-03, -3.92508324e-03,
         -3.62477516e-03, -3.34530085e-03, -3.08549435e-03,
         -2.84421476e-03, -2.62035292e-03, -2.41283640e-03,
         -2.22063325e-03, -2.04275460e-03, -1.87825636e-03,
         -1.72624014e-0

In [13]:
tt.names

['c',
 'i',
 'k',
 'y',
 'n0',
 's0',
 'theta0',
 'theta1',
 'theta2',
 'a0',
 'a1',
 'a2']

We will care about the irfs but these are also of type `oct2py.io.Struct`. We convert it as above. We will likley make this part of the above class later, but for now we create a new class. 

In [14]:
names2 = [attr for attr in dir(tt.irfs) if attr.endswith('shock')]
#names2 = [x for _, x in sorted(zip(tt.names, names2), key=lambda pair: pair[0][0])]

I will put more desciptors here later. However, it is getting the impulse responses as difference and levels. It is simultanosuly creating two dataframes containing all differences and levels data.  

In [15]:
#recall make_levels is created in simpler ffolder
import numpy as np
import pandas as np
class make_irfs:
	def __init__(self, dta, vars, names):
		self.data = dta
		self.vars = vars
		self.names = names
		self.create_difs()
		
  	
	def create_difs(self):
		for var in self.vars:
			setattr(self, var, pd.DataFrame())  # Initialize an empty DataFrame with the dynamic name
			list_short = [name for name in self.names if name.startswith(var)]
			for attr in list_short:
				x = getattr(self.data, attr)
				x = x.flatten()
				getattr(self, var)[attr] = x  # Assign the flattened array to the DataFrame
				new_columns = [col.replace(var + '_', '').replace('shock', '') for col in getattr(self, var).columns]
				getattr(self, var).columns = new_columns
    
    



In [16]:
varlist=['c','i','y','k','n0','s0']

In [17]:
ir = make_irfs(tt.irfs,varlist, names2)
print(ir.__dict__.keys())

dict_keys(['data', 'vars', 'names', 'c', 'i', 'y', 'k', 'n0', 's0'])


Lets get a sense of what we have. 

In [18]:
ir.y

,a0,a1,a2,theta0,theta1,theta2
0,0.065000,0.0,0.0,-0.033029,0.000000,0.000000
1,0.061364,0.0,0.0,-0.032785,-0.000308,-0.000308
2,0.057806,0.0,0.0,-0.032261,-0.000555,-0.000555
3,0.054347,0.0,0.0,-0.031515,-0.000750,-0.000750
4,0.051002,0.0,0.0,-0.030595,-0.000900,-0.000900
...,...,...,...,...,...,...
95,0.000017,0.0,0.0,-0.000016,-0.000001,-0.000001
96,0.000015,0.0,0.0,-0.000015,-0.000001,-0.000001
97,0.000014,0.0,0.0,-0.000013,-0.000001,-0.000001
98,0.000012,0.0,0.0,-0.000012,-0.000001,-0.000001


This will also need more documentation, but it is getting the results of dynare's simulation. It defines the individual series and also puts all in a dataframe. 

In [19]:
import numpy as np

class make_data:
	def __init__(self, dta,names):
		self.data = dta
		self.names = names
		self.make_levels()
		self.make_raw()

	def make_raw(self):
		self.df_raw = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			new_attr =attr + 'r'
			raw_level = self.data[j]
			setattr(self, new_attr, raw_level)
			self.df_raw[new_attr] = raw_level
			
  		
	def make_levels(self):
		self.df = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			raw_level = self.data[j]
			if attr != 'i' and 'theta' not in attr:
				current_level=np.exp(raw_level)
			else:
				current_level=raw_level
			setattr(self, attr, current_level)
			self.df[attr] = current_level  # Store in DataFrame



In [20]:
print(tt.endo_simul)

[[ 0.32335493  0.32749785  0.26155273 ...  0.6803493   0.59603283
   0.62131538]
 [ 0.69663611  0.71696416  0.62698374 ...  0.89237418  0.76224024
   0.73342646]
 [ 1.90189796  1.91812736  1.91984647 ...  2.20476839  2.20127316
   2.19388668]
 ...
 [ 0.05046192  0.03882792 -0.10821631 ...  0.43370307  0.24423277
   0.30235654]
 [ 0.05774259  0.01705476  0.08189624 ... -0.17029383 -0.28736746
  -0.15070018]
 [ 0.05409461  0.11390567  0.09518546 ...  0.24636957  0.15284976
   0.11729096]]


In [21]:
eg=make_data(tt.endo_simul,tt.names)

In [22]:
eg.df

,c,i,k,y,n0,s0,theta0,theta1,theta2,a0,a1,a2
0,1.381756,0.696636,6.698596,2.083050,0.615485,0.646760,0.371435,0.336633,0.357333,1.051757,1.059442,1.055584
1,1.387492,0.716964,6.808197,2.111079,0.614978,0.658563,0.379828,0.338713,0.353382,1.039592,1.017201,1.120646
2,1.298945,0.626984,6.819911,1.925344,0.614733,0.660417,0.378665,0.341832,0.344768,0.897433,1.085343,1.099863
3,1.246666,0.540226,6.744891,1.789389,0.624137,0.637214,0.362547,0.356065,0.348568,0.839455,1.135322,1.098793
4,1.195678,0.491697,6.630100,1.695404,0.607420,0.644512,0.365338,0.340224,0.331969,0.770931,1.227985,0.902043
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.837432,0.798063,8.827688,2.673459,0.638055,0.574989,0.349380,0.388848,0.412599,1.411090,0.995541,1.221073
9996,1.910710,0.838580,8.915999,2.804858,0.620783,0.588876,0.356715,0.367822,0.397671,1.483623,0.948270,1.304600
9997,1.974567,0.892374,9.068151,2.950220,0.606952,0.600659,0.368134,0.321529,0.425023,1.542961,0.843417,1.279372
9998,1.814904,0.762240,9.036511,2.602144,0.591883,0.592627,0.363731,0.309234,0.416988,1.276641,0.750236,1.165150


In [23]:
params.keys()

dict_keys(['eps', 'bet', 'delt', 'gamma0', 'gamman', 'vp', 'omega0', 'omega1', 'omega2', 'theta_var0', 'theta_var1', 'theta_var2', 'theta_rho0', 'theta_rho1', 'theta_rho2', 'theta_mean0', 'theta_mean1', 'theta_mean2', 'a_var0', 'a_var1', 'a_var2', 'a_rho0', 'a_rho1', 'a_rho2', 'a_mean0', 'a_mean1', 'a_mean2'])

In [24]:
M=eg.df
P=params
S=shocks

In [25]:
def x_find(g,t):
    thetag='theta'+str(g)
    ag='a'+str(g)
    omegag='omega'+str(g)
    thetai=M[thetag][t]
    theta0=M['theta0'][t]
    ai=M[ag][t]
    a0=M['a0'][t]
    k=M['k'][t-1]
    s0=M['s0'][t]
    n0=M['n0'][t]
    theta0=M['theta0'][t]
    eps=P['eps']
    omegai=P[omegag]
    omega0=P['omega0']
    
    x=(k*s0/n0)**((theta0-thetai)*(1-eps))*(omegai/omega0)**eps* \
    ((a0)**(1-theta0)/(ai)**(1-thetai)*(theta0/thetai)**thetai*((1-theta0)*(1-thetai))**(1-thetai))**(1-eps)
    
    return x

In [26]:
def X_find(t):
    X=0
    for g in range(1,n_goods):
        X+=x_find(g,t)
    return X

In [27]:
def Z_find(t):
    X=0
    for g in range(1,n_goods):
        thetag='theta'+str(g)
        thetai=M[thetag][t]
        X+=x_find(g,t)*thetai
    return X

In [28]:
eg.df['X'] = [X_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]
eg.df['Z'] = [Z_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]


In [29]:
eg.df

,c,i,k,y,n0,s0,theta0,theta1,theta2,a0,a1,a2,X,Z
0,1.381756,0.696636,6.698596,2.083050,0.615485,0.646760,0.371435,0.336633,0.357333,1.051757,1.059442,1.055584,NaN,NaN
1,1.387492,0.716964,6.808197,2.111079,0.614978,0.658563,0.379828,0.338713,0.353382,1.039592,1.017201,1.120646,1.550526,0.536254
2,1.298945,0.626984,6.819911,1.925344,0.614733,0.660417,0.378665,0.341832,0.344768,0.897433,1.085343,1.099863,1.474245,0.506099
3,1.246666,0.540226,6.744891,1.789389,0.624137,0.637214,0.362547,0.356065,0.348568,0.839455,1.135322,1.098793,1.391159,0.490072
4,1.195678,0.491697,6.630100,1.695404,0.607420,0.644512,0.365338,0.340224,0.331969,0.770931,1.227985,0.902043,1.421147,0.477306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.837432,0.798063,8.827688,2.673459,0.638055,0.574989,0.349380,0.388848,0.412599,1.411090,0.995541,1.221073,1.514220,0.605896
9996,1.910710,0.838580,8.915999,2.804858,0.620783,0.588876,0.356715,0.367822,0.397671,1.483623,0.948270,1.304600,1.591667,0.607477
9997,1.974567,0.892374,9.068151,2.950220,0.606952,0.600659,0.368134,0.321529,0.425023,1.542961,0.843417,1.279372,1.708229,0.624595
9998,1.814904,0.762240,9.036511,2.602144,0.591883,0.592627,0.363731,0.309234,0.416988,1.276641,0.750236,1.165150,1.687976,0.598433


In [34]:
eg.df['Q']=(eg.df['Z']-eg.df['theta0']*eg.df['X'])/eg.df['X']


In [35]:
eg.df

,c,i,k,y,n0,s0,theta0,theta1,theta2,a0,a1,a2,X,Z,Q
0,1.381756,0.696636,6.698596,2.083050,0.615485,0.646760,0.371435,0.336633,0.357333,1.051757,1.059442,1.055584,NaN,NaN,NaN
1,1.387492,0.716964,6.808197,2.111079,0.614978,0.658563,0.379828,0.338713,0.353382,1.039592,1.017201,1.120646,1.550526,0.536254,-0.033975
2,1.298945,0.626984,6.819911,1.925344,0.614733,0.660417,0.378665,0.341832,0.344768,0.897433,1.085343,1.099863,1.474245,0.506099,-0.035371
3,1.246666,0.540226,6.744891,1.789389,0.624137,0.637214,0.362547,0.356065,0.348568,0.839455,1.135322,1.098793,1.391159,0.490072,-0.010270
4,1.195678,0.491697,6.630100,1.695404,0.607420,0.644512,0.365338,0.340224,0.331969,0.770931,1.227985,0.902043,1.421147,0.477306,-0.029478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.837432,0.798063,8.827688,2.673459,0.638055,0.574989,0.349380,0.388848,0.412599,1.411090,0.995541,1.221073,1.514220,0.605896,0.050758
9996,1.910710,0.838580,8.915999,2.804858,0.620783,0.588876,0.356715,0.367822,0.397671,1.483623,0.948270,1.304600,1.591667,0.607477,0.024946
9997,1.974567,0.892374,9.068151,2.950220,0.606952,0.600659,0.368134,0.321529,0.425023,1.542961,0.843417,1.279372,1.708229,0.624595,-0.002495
9998,1.814904,0.762240,9.036511,2.602144,0.591883,0.592627,0.363731,0.309234,0.416988,1.276641,0.750236,1.165150,1.687976,0.598433,-0.009204


In [36]:
eg.df.describe()

,c,i,k,y,n0,s0,theta0,theta1,theta2,a0,a1,a2,X,Z,Q
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9999.000000,9999.000000,9999.000000
mean,1.370637,0.603405,6.797863,1.993632,0.614596,0.616660,0.352289,0.351301,0.352200,1.024466,1.017046,1.020203,1.529360,0.534218,-0.001797
std,0.271334,0.180062,1.318286,0.433999,0.032112,0.052608,0.040973,0.041045,0.039551,0.241614,0.241181,0.241632,0.164330,0.057047,0.050165
min,0.667451,-0.073593,3.369599,0.879101,0.513431,0.454503,0.210261,0.196717,0.193890,0.387068,0.468242,0.395064,1.023944,0.362329,-0.164916
25%,1.175605,0.482149,5.858672,1.684584,0.592106,0.579632,0.324577,0.323383,0.325865,0.853301,0.842323,0.849086,1.412822,0.494457,-0.036257
50%,1.343674,0.603239,6.661807,1.948032,0.613924,0.613600,0.352094,0.351299,0.353052,0.997489,0.990141,0.996582,1.519275,0.531457,-0.001603
75%,1.531543,0.725505,7.603314,2.252201,0.636118,0.651110,0.379948,0.379441,0.379177,1.163077,1.165578,1.166570,1.638180,0.571613,0.032961
max,2.583597,1.209831,12.228828,4.067316,0.758383,0.829589,0.491843,0.493939,0.499308,2.539666,2.630144,2.417524,2.277025,0.774578,0.175906


In [ ]:

# Function to highlight the diagonal
def highlight_diagonal(data):
	attr = 'background-color: yellow'
	# Create a DataFrame with empty strings
	df_styler = pd.DataFrame('', index=data.index, columns=data.columns)
	# Set the diagonal to the desired attribute
	np.fill_diagonal(df_styler.values, attr)
	return df_styler

# Apply the function to the DataFrame
df_cov = df_cov.style.apply(highlight_diagonal, axis=None)

# Display the styled DataFrame
df_cov